In [171]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import csv
import os
import dedupe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from future.builtins import next
from unidecode import unidecode
import recordlinkage

In [172]:
df=pd.read_csv("Deduplication Problem - Sample Dataset.csv")

In [173]:
df

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM
5,BLAND JR,21/02/62,F,WILLIAM
6,BLAND JR,21/02/62,F,WILLIAM
7,BLAND JR,21/02/62,F,WILLIAM
8,SHAFFER JR,21/02/62,F,WILLIAM
9,SHAFFER JR,21/02/62,F,WILLIAM


## Removing Exact duplicates

In [174]:
df1=df.drop_duplicates(keep='first')
df1['day'],df1['month']=df1['dob'].str.split('/', 1).str
df1['month'],df1['year']=df1['month'].str.split('/', 1).str
df1=df1.drop(['dob'],axis=1)
df1[['day','month','year']] = df1[['day','month','year']].apply(pd.to_numeric)

In [175]:
#Approach 1
df1['fullname']=df1['fn']+" "+df1['ln']
df1=df1.sort_values(by=['day','month','year','gn'])
df1=df1.reset_index(drop=True)

In [176]:
df1

,ln,gn,fn,day,month,year,fullname
0,SMITH JR,F,WILLIAM,1,3,68,WILLIAM SMITH JR
1,ROTHMEYER JR,F,WILLIAM,1,3,68,WILLIAM ROTHMEYER JR
2,ASBY JR,F,WILLIAM,1,3,68,WILLIAM ASBY JR
3,SALTER JR,F,WILLIAM,1,3,68,WILLIAM SALTER JR
4,MICHAELSON JR,M,ROY,1,5,51,ROY MICHAELSON JR
5,FUNARO JONES,F,HARRIET,6,12,37,HARRIET FUNARO JONES
6,FUNARO J,F,HARRIET,6,12,37,HARRIET FUNARO J
7,FUNARO,F,HARRIET,6,12,37,HARRIET FUNARO
8,GRIFFIN JR,M,DARL,7,5,37,DARL GRIFFIN JR
9,GRIFFIN JR,M,CLIFFORD,7,5,37,CLIFFORD GRIFFIN JR


### Finding nearby duplicates for a string using fuzzy matching

In [177]:
def pros(df1):
    FULL_MATCHING_THRESHOLD = 80
    PARTIAL_MATCHING_THRESHOLD = 100
    SORT_MATCHING_THRESHOLD = 100
    TOKEN_MATCHING_THRESHOLD = 100
    MAX_MATCHES=4
    current_db_dataframe=df1
    
    def find_matches(matchThis):
        rows = current_db_dataframe['fullname'].values.tolist();
        rows.remove(matchThis)
        matches= process.extractBests(matchThis,rows,scorer=fuzz.ratio,score_cutoff=FULL_MATCHING_THRESHOLD,limit=MAX_MATCHES)
        if len(matches)==0:
            matches= process.extractBests(matchThis,rows,scorer=fuzz.partial_ratio,score_cutoff=PARTIAL_MATCHING_THRESHOLD,limit=MAX_MATCHES);
            if len(matches)==0:
                matches= process.extractBests(matchThis,rows,scorer=fuzz.token_set_ratio,score_cutoff=TOKEN_MATCHING_THRESHOLD,limit=MAX_MATCHES);
                if len(matches)==0:
                    matches= process.extractBests(matchThis,rows,scorer=fuzz.token_sort_ratio,score_cutoff=SORT_MATCHING_THRESHOLD,limit=MAX_MATCHES);
        return matches[0][0] if len(matches)>0 else None


    fn_find_matches = lambda x: find_matches(x)
    current_db_dataframe['Duplicate']=current_db_dataframe.applymap(fn_find_matches)
    current_db_dataframe.to_csv("using-fuzzy.csv", index=False) #saving results to external file

In [178]:
pros(df1[["fullname"]])

In [179]:
result=pd.read_csv("using-fuzzy.csv")

### Grouping
   To reduce comparison between pairs of observation, grouped on three basis -
   - First they are sorted with respect to dob for smoothly grouping.
   - For duplicate observation, dob and gender must be same.
   - First name is exactly same.
    

In [180]:
#Grouping
def grouping(df1):
    index_ofgroup_list=[]
    st=0
    end=0
    total=df1.count(axis=0)[0]
    for i in range(1,total):
        end+=1
        name1=df1.loc[st,["fn"]][0]
        name2=df1.loc[i,["fn"]][0]
        if(name1.find(" ")!=-1):
            name1=name1[0:name1.find(" ")]
        if(name2.find(" ")!=-1):
            name2=name2[0:name2.find(" ")]

        if(name1!=name2):
            index_ofgroup_list.append([st,end])
            st=end
            if(end==total-1):
                index_ofgroup_list.append([st,end+1])
            #print(st,end)
        elif(df1.loc[i,["day"]][0]!=df1.loc[st,["day"]][0] or df1.loc[i,["month"]][0]!=df1.loc[st,["month"]][0] or
            df1.loc[i,["year"]][0]!=df1.loc[st,["year"]][0] or df1.loc[i,["gn"]][0]!=df1.loc[st,["gn"]][0]):
            index_ofgroup_list.append([st,end])
            st=end
            if(end==total-1):
                index_ofgroup_list.append([st,end+1])
    return index_ofgroup_list
                #print(st,end)

In [181]:

index_ofgroup_list=grouping(df1)

## Solution 1 by fuzzy matching in strings

In [182]:
def approach1(df1):
    df1["stay"]=True
    for index in index_ofgroup_list:
        for i in range(index[0],index[1]):
            for j in range(i+1,index[1]):
                fn1=df1.loc[i,["fullname"]][0]
                fn2=df1.loc[j,["fullname"]][0]
                if(fuzz.token_set_ratio(fn1,fn2)>90):
                    df1.set_value(j,"stay",False)
    
    df1=df1[df1["stay"]==True]
    df1=df1.reset_index(drop=True)
    df1=df1.drop(["stay"],axis=1)
    return df1
df2=approach1(df1)
dfa=df2.copy()
dfa['dob']=dfa['day'].astype(str)+'/'+dfa['month'].astype(str)+'/'+dfa['year'].astype(str)
dfa=dfa.drop(['day','month','year','fullname'],axis=1)
dfa.to_csv(os.getcwd()+'/Fuzzy.csv')

## Solution 2 - Training with pairs of observations

In [185]:
def Compare(f_1, f_2):
    if f_1 and f_2 :
        f_1=str(f_1).split('/')
        f_2=str(f_2).split('/')

        if f_1[0] == f_2[0] and f_1[1] == f_2[1] and f_1[2] == f_2[2] :
            return 0
        return 1

In [154]:
training_file = 'csv_example_training.json'
variables = [{'field' : 'ln', 'type': 'String'},
               {'field' : 'dob', 'type': 'Custom', 'comparator' : Compare },
#              {'field' : 'day','type' : "Exact"},
#              {'field' : 'month','type' : "Exact"},
#              {'field' : 'year','type' : "Exact"},
             {'field' : 'gn', 'type': 'Categorical','categories':["M","F"]},
            {'field' : 'fn', 'type': 'String'},

             ]
Deduper = dedupe.Dedupe(variables)
dt=df.T.to_dict()

### Labeling on the basis of results obtained from fuzzy matching

In [155]:
match=[]
distinct=[]
key=0

for i in range(len(dt)):
    for j in range(len(dt)):
        if(i!=j):
            fn1=dt[i]["fn"]+dt[i]["ln"]
            fn2=dt[j]["fn"]+dt[j]["ln"]
            if(fuzz.token_set_ratio(fn1,fn2)>90 and dt[i]['dob']==dt[j]['dob'] and
                dt[i]['gn']==dt[j]['gn']):
                match.append((dt[i],dt[j]))
            elif(75<fuzz.token_set_ratio(fn1,fn2) and fuzz.token_set_ratio(fn1,fn2)<90 or 
                dt[i]['dob']!=dt[j]['dob'] or dt[i]['gn']!=dt[j]['gn']):
                distinct.append((dt[i],dt[j]))

In [157]:
print(len(match),len(distinct))

(214, 9810)


In [158]:
distinct=distinct[0:3*len(match)]

In [159]:
labeled_eg = {'match':match,'distinct':distinct}

### Using Classifiers 
   They can be easily modified by replacing the Deduper classifier with required accordingly.
   Note - The replaced classifier must have two methods fit and predict proba

In [164]:
# from sklearn.gaussian_process import GaussianProcessClassifier
# Deduper.classifier=GaussianProcessClassifier()
from sklearn.ensemble import RandomForestClassifier
Deduper.classifier=RandomForestClassifier(max_depth=8,random_state=0)

In [160]:
Deduper.sample(dt,15000)


In [161]:
Deduper.markPairs(labeled_eg)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(LevenshteinCanopyPredicate: (2, fn), SimplePredicate: (sameFiveCharStartPredicate, ln))


In [162]:
Deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.981858956839
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(LevenshteinCanopyPredicate: (2, fn), TfidfNGramCanopyPredicate: (0.8, ln))


In [110]:
print('reading labeled examples from ', training_file)
with open(os.getcwd()+'/'+training_file, 'w') as f:
    Deduper.writeTraining(f)

('reading labeled examples from ', 'csv_example_training.json')


In [163]:
threshold = Deduper.threshold(dt, recall_weight=1)

INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.878
INFO:dedupe.api:precision: 0.998
INFO:dedupe.api:With threshold: 0.973


### Duplicate sets and Cluster ID's with predicted probability for the pair observations

In [164]:
print('clustering...')
clustered_dupes = Deduper.match(dt, threshold)

print('# duplicate sets', len(clustered_dupes))

clustering...
('# duplicate sets', 29)


In [165]:
df.to_csv(os.getcwd()+'/'+'input.csv')

### Saving output files

In [166]:
input_file=os.getcwd()+'/'+'input.csv'
output_file=os.getcwd()+'/'+'output.csv'
final_file=os.getcwd()+'/csv_final_output.csv'
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [dt[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

singleton_id = cluster_id + 1

with open(output_file, 'w') as f_output, open(input_file,'rU') as f_input:
    writer = csv.writer(f_output)
    reader = csv.reader(f_input)
    
    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')
    canonical_keys = canonical_rep.keys()
    for key in canonical_keys:
        heading_row.append('canonical_' + key)

    writer.writerow(heading_row)

    for row in reader:
        row_id = int(row[0])
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            canonical_rep = cluster_membership[row_id]["canonical representation"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            for key in canonical_keys:
                row.append(canonical_rep[key].encode('utf8'))
        else:
            row.insert(0, None)
            row.insert(0, singleton_id)
            singleton_id += 1
            for key in canonical_keys:
                row.append(None)
        writer.writerow(row)

f_output.close()
with open(output_file, 'r') as f_output, open(final_file,'w') as fin_output:
    reader = csv.reader(f_output)
    writer = csv.writer(fin_output)
    head_row=["ID","ln","dob","gn","fn"]
    writer.writerow(head_row)
    uniClusterId=list()
    i=1
    for row in reader:
        if i==1:
            i+=1
            continue
        if row[0] in uniClusterId:
            continue
        else:
            uniClusterId.append(row[0])
            mid_row=(row[2],row[3],row[4],row[5],row[6])
            writer.writerow(mid_row)